Dans ce notebook, je cherche à coder un algorithme qui permet de traduire une phrase simple en français vers l'anglais. Je me base sur ma compréhension de cette vidéo https://www.youtube.com/watch?v=FWQ2SvE-nbM et sur la décomposition des étapes par ChatGPT 4o. 

## Étape 1 : Créer un jeu de données avec des associations phrase en Anglais / phrase en Français. 

In [62]:
pairs = [
    ("je t'aime", "i love you"),
    ("j'aime les pâtes", "i love pasta"),
    ("bonjour", "hello"),
    ("je suis fatigué", "i am tired"),
]


## Étape 2 : Encoder les phrases en indices

Pour coder cette deuxième étape, j'utilise le document colab en description de la vidéo cf. l'url du début.

In [65]:
#On installe tensor flow
!pip install tensorflow

In [66]:
import tensorflow as tf

In [67]:
#On importe la fonction one_hot qui nous permet d'associer à un mot un vecteur de la base canonique de dimension la taille du vocabulaire qu'on a
# en bref base canonique de dim = voc_size
from tensorflow.keras.preprocessing.text import one_hot

In [68]:
#voc_size : de ce que j'ai compris on met ce qu'on veut on est pas obligé de prendre le nombre de mot exact qu'on a dans notre bdd
voc_size=500

In [69]:
#One-hot representation : on utilise la fonction de tensor-flow
#Là on vient prendre les phrases de chaque paire et les mots de chaque phrase 
onehot_repr=[one_hot(words,voc_size) for vec in pairs for words in vec]
print(onehot_repr)

[[185, 151], [271, 117, 234], [267, 436, 304], [271, 117, 187], [116], [54], [185, 63, 196], [271, 257, 431]]


J'ai l'impression qu'il va y avoir deux problèmes ici : 
- Un évoqué dans la vidéo c'est celui de la taille de nos vecteurs il faut que ce soit les mêmes pour pouvoir entraîner notre modèle mais ça j'ai déjà la solution vu que c'est dans la vidéo
- L'autre problème pour moi c'est que vu qu'on dissocie nos paires dans le one-hot matching je pense qu'on perd l'équivalence entre les phrases. Pour ce second problème, je ne le considère pas pour le moment mais je garde en tête que ça risque d'être embêtant à un moment.
  

In [71]:
#On fait tout pareil que Krish
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [72]:
import numpy as np

In [73]:
## pre padding : j'ai pas trop compris l'intérêt d'utiliser du pre-padding plutôt que post-padding et inversement donc on part sur pre padding
sent_length=5 #taille des vecteurs que je vais avoir 
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) #on ajoute des 0 avant la séquence one-hot pour avoir un vecteur de la bonne taille 
print(embedded_docs) #Maintenant on constate que tout le monde à la même taille

[[  0   0   0 185 151]
 [  0   0 271 117 234]
 [  0   0 267 436 304]
 [  0   0 271 117 187]
 [  0   0   0   0 116]
 [  0   0   0   0  54]
 [  0   0 185  63 196]
 [  0   0 271 257 431]]


In [74]:
## 10 feature dimesnions
# en gros je choisis le nombre de dimensions que je veux donner à chaque token
dim=10

In [107]:
# Là j'utilise tout comme Krish
model=Sequential() #ça ça me permet de créer un modèle où je vais pouvoir empiler différentes couches qui prennent 1 entrée et 1 sortie et tq chaque couches se succèdent
model.add(Embedding(voc_size,dim)) #là j'ajoute ma couche de Embedding avec le nombre de dimensions
model.compile('adam','mse') # "adam" c'est ce qui ajuste les poids pendant l'entraînement "mse" c'est ma mean squared error que j'utilise pour la régression
model.build(input_shape=(None, sent_length)) # c'est pour remplacer le input_shape du code d'origine qui ne fonctionnait plus pour cette version de tensor flow

In [103]:
model.summary()
#J'avais un problème ici car mon modèle ne s'était pas créé c'était dû au input shape qui manquait de ce que je comprends c'est des histoires de versions de tensor flow
#J'ai rajouté la 4ème ligne dans la cellule précédente

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [95]:
#"je t'aime"
embedded_docs[0]

array([  0,   0,   0, 185, 151], dtype=int32)

In [110]:
model.predict(np.array([embedded_docs[0]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


array([[[ 0.04170972, -0.01308879,  0.04921211, -0.04347448,
         -0.00993432,  0.0307938 ,  0.03554085,  0.0374962 ,
         -0.01803453, -0.00540574],
        [ 0.04170972, -0.01308879,  0.04921211, -0.04347448,
         -0.00993432,  0.0307938 ,  0.03554085,  0.0374962 ,
         -0.01803453, -0.00540574],
        [ 0.04170972, -0.01308879,  0.04921211, -0.04347448,
         -0.00993432,  0.0307938 ,  0.03554085,  0.0374962 ,
         -0.01803453, -0.00540574],
        [ 0.03508558, -0.00691645,  0.04332891, -0.02643063,
          0.02824534, -0.03233401, -0.00267701,  0.01194036,
         -0.01221845,  0.0302863 ],
        [-0.04643903,  0.04642668, -0.02902323,  0.0206237 ,
         -0.01432235, -0.03569045, -0.0480106 , -0.03034617,
         -0.00200113,  0.00768837]]], dtype=float32)

In [125]:
encoded_total = model.predict(np.array(embedded_docs)) #toutes nos phrases traduites en matrices
print(encoded_total)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[[ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.03508558 -0.00691645  0.04332891 -0.02643063  0.02824534
   -0.03233401 -0.00267701  0.01194036 -0.01221845  0.0302863 ]
  [-0.04643903  0.04642668 -0.02902323  0.0206237  -0.01432235
   -0.03569045 -0.0480106  -0.03034617 -0.00200113  0.00768837]]

 [[ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.00776789 -0.01692021  0.04038301  0.0281355  -0.04011698
    0.02

On revient au second problème que j'évoquais précédemment : en fait ce n'est pas très grave d'avoir destructurée mes paires, maintenant ce que je vais faire c'est simplement séparer en encoder vs decoder les phrases selon mes indices (pair --> encoder (phrase française), impair --> decoder (phrase anglaise) )

In [123]:
#On sépare les phrases françaises des phrases anglaises (on suppose donc que l'embedding n'a pas modifié l'ordre ce qui devrait être le cas) 
encoder_input = encoded_total[::2]  # 0, 2, 4, 6 → français
decoder_input = encoded_total[1::2]  # 1, 3, 5, 7 → anglais
print(encoder_input) 

[[[ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.03508558 -0.00691645  0.04332891 -0.02643063  0.02824534
   -0.03233401 -0.00267701  0.01194036 -0.01221845  0.0302863 ]
  [-0.04643903  0.04642668 -0.02902323  0.0206237  -0.01432235
   -0.03569045 -0.0480106  -0.03034617 -0.00200113  0.00768837]]

 [[ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.04170972 -0.01308879  0.04921211 -0.04347448 -0.00993432
    0.0307938   0.03554085  0.0374962  -0.01803453 -0.00540574]
  [ 0.01635173  0.01477155 -0.02947416  0.0272986  -0.0212079
    0.02891922  0.01252587 -0.01019554  0.04921

Je pense avoir compris tout ce qui précède là je fais un copié-collé de Chat pour construire mes modèles encodeur decodeur et comprendre un peu mieux comment ça marche. 

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Paramètres
vocab_size = 500
embedding_dim = 10
latent_dim = 64
sent_length = 5  # nombre de tokens par phrase

# --- Encodeur ---
encoder_inputs = Input(shape=(sent_length,))  # on passe des indices
enc_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# --- Décodeur ---
decoder_inputs = Input(shape=(sent_length,))  # indices aussi
dec_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# --- Modèle complet ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 5, 10)     │      5,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 5, 10)     │      5,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 64),      │     19,200 │ embedding[0][0]   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 5, 64),   │     19,200 │ embedding_1[0][0… │
│                     │ (None, 64),       │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5, 500)    │     32,500 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 80,900 (316.02 KB)

 Trainable params: 80,900 (316.02 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Ici, pas d'Embedding manuelle avant
model.fit([padded_fr, decoder_input], decoder_target, epochs=100)


NameError: name 'padded_fr' is not defined

Le problème ici c'est que je n'ai pas séparé dès le début mes phrases françaises de mes phrases anglaises. Donc j'ai des petits problèmes ici. Je fais une version corrigé où j'explique ce qui fonctionne pas et ce qui diffère de ce que j'ai fait.